In [1]:
import tensorflow_hub as hub
import numpy as np
import tensorflow_text
from simpleneighbors import SimpleNeighbors

In [9]:
en_sen = ["home", "i like walk around my home"]
ru_sen = ["дом", "я люблю гулять вокруг моего дома"]

In [2]:
embed = hub.load("/home/araytsyn/Downloads/encoder_model")

In [10]:
en_result = embed(en_sen)
ru_result = embed(ru_sen)

In [8]:
en_result

<tf.Tensor: shape=(3, 512), dtype=float32, numpy=
array([[ 0.10093052, -0.03811248,  0.01992672, ...,  0.06740665,
         0.01838608,  0.05333166],
       [ 0.0364161 , -0.03177984,  0.0666375 , ...,  0.03586746,
        -0.001629  ,  0.04395662],
       [-0.00552387,  0.0059953 , -0.01480542, ..., -0.0398063 ,
         0.03113597, -0.03667422]], dtype=float32)>

In [11]:
similarity_matrix = np.inner(en_result, ru_result)

In [12]:
similarity_matrix

array([[0.8754536 , 0.37812936],
       [0.26897886, 0.76994395]], dtype=float32)

In [23]:
data_en = ["i want a burger", "i want to play football", "i like to walk with the cat"]
data_ru = ["я хочу бургер", "я хочу поиграть в футбол", "я люблю гулять с кошкой"]

annoy_index_en = SimpleNeighbors(512, metric='dot')
for i, item in enumerate(data_en):
    emb = embed([item,]).numpy()[0]
    annoy_index_en.add_one(item, emb)
annoy_index_en.build()

In [26]:
annoy_index_en.nearest(embed(["i would like walk",])[0])

['i like to walk with the cat', 'i want to play football', 'i want a burger']

In [28]:
annoy_index_ru = SimpleNeighbors(512, metric='dot')
for i, item in enumerate(data_ru):
    emb = embed([item,]).numpy()[0]
    annoy_index_ru.add_one(item, emb)
annoy_index_ru.build()

In [30]:
annoy_index_ru.nearest(embed(["я играть в хоккей",])[0])

['я хочу поиграть в футбол', 'я люблю гулять с кошкой', 'я хочу бургер']

In [3]:
import pandas as pd
df = pd.read_csv(r'/home/araytsyn/Documents/quora_duplicate_questions.tsv', sep='\t')

In [4]:
df

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...,...,...,...
404285,404285,433578,379845,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404286,404286,18840,155606,Do you believe there is life after death?,Is it true that there is life after death?,1
404287,404287,537928,537929,What is one coin?,What's this coin?,0
404288,404288,537930,537931,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0


In [5]:
df = df.drop(df[df.is_duplicate == 0].index)

In [6]:
df

,id,qid1,qid2,question1,question2,is_duplicate
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
11,11,23,24,How do I read and find my YouTube comments?,How can I see all my Youtube comments?,1
12,12,25,26,What can make Physics easy to learn?,How can you make physics easy to learn?,1
13,13,27,28,What was your first sexual experience like?,What was your first sexual experience?,1
...,...,...,...,...,...,...
404280,404280,537922,537923,What are some outfit ideas to wear to a frat p...,What are some outfit ideas wear to a frat them...,1
404281,404281,99131,81495,Why is Manaphy childish in Pokémon Ranger and ...,Why is Manaphy annoying in Pokemon ranger and ...,1
404282,404282,1931,16773,How does a long distance relationship work?,How are long distance relationships maintained?,1
404284,404284,537926,537927,What does Jainism say about homosexuality?,What does Jainism say about Gays and Homosexua...,1


In [7]:
questions1 = df['question1'].to_numpy()[:10000]
questions2 = df['question2'].to_numpy()[:10000]

In [8]:
annoy_index_questions1 = SimpleNeighbors(512, metric='dot')
for i, item in enumerate(questions1):
    emb = embed([item,]).numpy()[0]
    annoy_index_questions1.add_one(item, emb)
annoy_index_questions1.build()

In [14]:
annoy_index_questions1.nearest(embed([questions2[1],])[0])

['How can I be a good geologist?',
 'How can I be a good software engineer?',
 'How do I become a great computer scientist?',
 'What should I do after computer science engineering?',
 'How can I become a true computer science engineer?',
 'As a computer science student, how can I get good internships?',
 'How can I find my passion career?',
 'What is the best way to learn astronomy?',
 'How do I study for gre?',
 'What should I know to get into GSoC?',
 'How can I find my path and my dream job?',
 'How do I become better at research?']

In [17]:
good_counter = 0
for i, question_test in enumerate(questions2):
    ret = annoy_index_questions1.nearest(embed([question_test,])[0])[0]
    if ret == questions1[i]:
        good_counter += 1
print(good_counter / len(questions1))

0.5024
